# Olist 데이터 전처리 파이프라인

## 목표
1. 불완전 기간 데이터 제외 (2016년 9-12월, 2018년 9월 이후)
2. 브라질 특수 기간 데이터 제외 (파업, 연휴 등 외부 요인)
   - 우편국 파업 2017 (9/19 ~ 10월)
   - 세무관 파업 2017 (11/1 ~ 12월)
   - 카니발 2017 (2/24 ~ 3/5)
   - 카니발 2018 (2/9 ~ 2/18)
   - 트럭 파업 2018 (5/21 ~ 5/31)
3. 배송 시점 이상 데이터 제거
4. IQR 기준 이상치 제거 (PG사/물류사 책임구간)
5. 결측치 제거
6. 모든 데이터셋 통합

In [236]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = 'Olist_DataSet/'

## 1. 데이터 로드 및 기간 필터링


In [237]:
orders = pd.read_csv(f'{DATA_PATH}olist_orders_dataset.csv')
order_items = pd.read_csv(f'{DATA_PATH}olist_order_items_dataset.csv')
sellers = pd.read_csv(f'{DATA_PATH}olist_sellers_dataset.csv')
products = pd.read_csv(f'{DATA_PATH}olist_products_dataset.csv')
reviews = pd.read_csv(f'{DATA_PATH}order_reviews.csv')
customers = pd.read_csv(f'{DATA_PATH}olist_customers_dataset.csv')
payments = pd.read_csv(f'{DATA_PATH}olist_order_payments_dataset.csv')
geolocation = pd.read_csv(f'{DATA_PATH}olist_geolocation_dataset.csv')
category_trans = pd.read_csv(f'{DATA_PATH}product_category_name_translation.csv')


In [238]:
# 날짜 컬럼 변환
date_cols = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date',
             'order_estimated_delivery_date']

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col])


In [239]:
# 불완전 기간 데이터 제외 (2016년 9-12월, 2018년 9월 이후)
exclude_incomplete = (
    ((orders['order_purchase_timestamp'].dt.year == 2016) & 
     (orders['order_purchase_timestamp'].dt.month >= 9)) |
    ((orders['order_purchase_timestamp'].dt.year == 2018) & 
     (orders['order_purchase_timestamp'].dt.month >= 9))
)

# 브라질 특수 기간 제외 
# 1. 우편국 파업 2017 (9월 19일 ~ 10월 말)
postal_strike_2017 = (
    (orders['order_purchase_timestamp'] >= '2017-09-19') & 
    (orders['order_purchase_timestamp'] < '2017-11-01')
)

# 2. 세무관 파업 2017 (11월 1일 ~ 12월 말) - 수입 통관 지연
customs_strike_2017 = (
    (orders['order_purchase_timestamp'] >= '2017-11-01') & 
    (orders['order_purchase_timestamp'] < '2018-01-01')
)

# 3. 카니발 2017 (2월 24일 ~ 3월 5일)
carnival_2017 = (
    (orders['order_purchase_timestamp'] >= '2017-02-24') & 
    (orders['order_purchase_timestamp'] < '2017-03-06')
)

# 4. 카니발 2018 (2월 9일 ~ 2월 18일)
carnival_2018 = (
    (orders['order_purchase_timestamp'] >= '2018-02-09') & 
    (orders['order_purchase_timestamp'] < '2018-02-19')
)

# 5. 트럭 파업 2018 (5월 21일 ~ 5월 31일) - 국가적 물류 마비
truck_strike_2018 = (
    (orders['order_purchase_timestamp'] >= '2018-05-21') & 
    (orders['order_purchase_timestamp'] < '2018-06-01')
)

# 모든 제외 조건 통합
exclude_periods = (
    exclude_incomplete |
    postal_strike_2017 |
    customs_strike_2017 |
    carnival_2017 |
    carnival_2018 |
    truck_strike_2018
)

before_count = len(orders)
orders = orders[~exclude_periods].copy()
after_count = len(orders)

print(f"제외된 주문 수: {before_count - after_count:,}건")
print(f"  - 불완전 기간 (2016년 9-12월, 2018년 9월 이후)")
print(f"  - 우편국 파업 2017 (9/19 ~ 10월)")
print(f"  - 세무관 파업 2017 (11/1 ~ 12월)")
print(f"  - 카니발 2017 (2/24 ~ 3/5)")
print(f"  - 카니발 2018 (2/9 ~ 2/18)")
print(f"  - 트럭 파업 2018 (5/21 ~ 5/31)")
print(f"남은 주문 수: {after_count:,}건")


제외된 주문 수: 24,220건
  - 불완전 기간 (2016년 9-12월, 2018년 9월 이후)
  - 우편국 파업 2017 (9/19 ~ 10월)
  - 세무관 파업 2017 (11/1 ~ 12월)
  - 카니발 2017 (2/24 ~ 3/5)
  - 카니발 2018 (2/9 ~ 2/18)
  - 트럭 파업 2018 (5/21 ~ 5/31)
남은 주문 수: 75,221건


In [240]:
# 이상 데이터 탐지
anomalies = {}

# 1. 운송업체 인수일 > 고객 배송 완료일
anomalies['1. 운송업체 인수일 > 고객 배송일'] = orders[
    (orders['order_delivered_carrier_date'].notna()) & 
    (orders['order_delivered_customer_date'].notna()) &
    (orders['order_delivered_carrier_date'] > orders['order_delivered_customer_date'])
]

# 2. 주문 승인일 > 운송업체 인수일
anomalies['2. 주문 승인일 > 운송업체 인수일'] = orders[
    (orders['order_approved_at'].notna()) & 
    (orders['order_delivered_carrier_date'].notna()) &
    (orders['order_approved_at'] > orders['order_delivered_carrier_date'])
]

# 3. 배송완료 상태인데 배송일 없음
anomalies['3. 배송완료 상태인데 배송일 없음'] = orders[
    (orders['order_status'] == 'delivered') & 
    (orders['order_delivered_customer_date'].isna())
]

# 4. 미배송 상태인데 배송일 존재
anomalies['4. 미배송 상태인데 배송일 존재'] = orders[
    (orders['order_status'] != 'delivered') & 
    (orders['order_delivered_customer_date'].notna())
]


## 2. 배송 시점 이상 데이터 제거


In [241]:
# 이상 데이터 제거
all_anomaly_ids = set()
for name, df in anomalies.items():
    all_anomaly_ids.update(df['order_id'].tolist())
orders = orders[~orders['order_id'].isin(all_anomaly_ids)].copy()

print(f"이상 데이터 제외 후 주문 수: {len(orders):,}건")


이상 데이터 제외 후 주문 수: 73,868건


## 3. IQR 기준 이상치 제거

- **PG사 책임구간**: `order_approved_at - order_purchase_timestamp`
- **물류사 책임구간**: `order_delivered_customer_date - order_delivered_carrier_date`
- **제거 조건**: 음수 또는 Q3 + 1.5*IQR 초과


In [242]:
# PG사 책임구간: order_approved_at - order_purchase_timestamp
# 물류사 책임구간: order_delivered_customer_date - order_delivered_carrier_date

# 시간 차이 계산 (일 단위)
orders['pg_time'] = (
    orders['order_approved_at'] - orders['order_purchase_timestamp']
).dt.total_seconds() / (24 * 60 * 60)

orders['logistics_time'] = (
    orders['order_delivered_customer_date'] - orders['order_delivered_carrier_date']
).dt.total_seconds() / (24 * 60 * 60)

# IQR 적용을 위한 추가 정보 조인

# 1. 결제수단 정보 조인 (PG사 책임구간용)
# 한 주문에 여러 결제수단이 있을 수 있으므로 대표 결제수단 선택 (첫 번째)
payment_type_df = payments.groupby('order_id')['payment_type'].first().reset_index()
orders = orders.merge(payment_type_df, on='order_id', how='left')

In [243]:
# 2. 카테고리 정보 조인 (물류사 책임구간용)
# order_items -> products 조인하여 카테고리 가져오기
order_category = order_items[['order_id', 'product_id']].drop_duplicates('order_id')
order_category = order_category.merge(products[['product_id', 'product_category_name']], on='product_id', how='left')
orders = orders.merge(order_category[['order_id', 'product_category_name']], on='order_id', how='left')


In [244]:
def remove_iqr_outliers_by_group(df, column, group_col):
    """그룹별 IQR 기준으로 이상치 제거 (음수 및 Q3+1.5*IQR 초과)"""
    keep_mask = pd.Series(True, index=df.index)
    
    for group_name, group_df in df.groupby(group_col):
        if pd.isna(group_name):
            continue
        valid_data = group_df[column].dropna()
        if len(valid_data) < 4:
            continue
        
        Q1, Q3 = valid_data.quantile(0.25), valid_data.quantile(0.75)
        upper_bound = Q3 + 1.5 * (Q3 - Q1)
        outlier_mask = (group_df[column] < 0) | (group_df[column] > upper_bound)
        keep_mask.loc[group_df.index] = ~outlier_mask | group_df[column].isna()
    
    return keep_mask


In [245]:
# PG사 책임구간 이상치 제거 (결제수단별 IQR)
pg_mask = remove_iqr_outliers_by_group(orders, 'pg_time', 'payment_type')
orders = orders[pg_mask].copy()

In [246]:
# 물류사 책임구간 이상치 제거 (카테고리별 IQR)
logistics_mask = remove_iqr_outliers_by_group(orders, 'logistics_time', 'product_category_name')
orders = orders[logistics_mask].copy()

In [247]:
# 임시 컬럼 제거 및 유효한 order_id 목록
orders = orders.drop(columns=['pg_time', 'logistics_time', 'payment_type', 'product_category_name'])
valid_order_ids = orders['order_id'].unique()


In [248]:
# 각 데이터셋 결측치 확인
datasets = {
    'orders': orders,
    'order_items': order_items,
    'sellers': sellers,
    'products': products,
    'reviews': reviews,
    'customers': customers,
    'payments': payments,
    'category_trans': category_trans
}

print("=" * 60)
print("각 데이터셋 결측치 현황")
print("=" * 60)

for name, df in datasets.items():
    null_count = df.isnull().sum().sum()
    null_rows = df.isnull().any(axis=1).sum()
    print(f"\n{name}:")
    print(f"  전체 행: {len(df):,}, 결측치 포함 행: {null_rows:,}")
    if null_count > 0:
        null_cols = df.isnull().sum()
        null_cols = null_cols[null_cols > 0]
        for col, cnt in null_cols.items():
            print(f"    - {col}: {cnt:,}건")


각 데이터셋 결측치 현황

orders:
  전체 행: 58,746, 결측치 포함 행: 1,732
    - order_approved_at: 106건
    - order_delivered_carrier_date: 971건
    - order_delivered_customer_date: 1,718건

order_items:
  전체 행: 112,650, 결측치 포함 행: 0

sellers:
  전체 행: 3,095, 결측치 포함 행: 0

products:
  전체 행: 32,951, 결측치 포함 행: 611
    - product_category_name: 610건
    - product_name_lenght: 610건
    - product_description_lenght: 610건
    - product_photos_qty: 610건
    - product_weight_g: 2건
    - product_length_cm: 2건
    - product_height_cm: 2건
    - product_width_cm: 2건

reviews:
  전체 행: 99,224, 결측치 포함 행: 56,518
    - review_comment_message: 56,518건

customers:
  전체 행: 99,441, 결측치 포함 행: 0

payments:
  전체 행: 103,886, 결측치 포함 행: 0

category_trans:
  전체 행: 73, 결측치 포함 행: 0


In [249]:
# 결측치 제거 (reviews는 제외 - review_score 활용)
orders = orders.dropna()
order_items = order_items.dropna()
sellers = sellers.dropna()
products = products.dropna()
customers = customers.dropna()
payments = payments.dropna()
category_trans = category_trans.dropna()


In [250]:
# 유효한 order_id로 필터링 (orders 기준)
valid_order_ids = orders['order_id'].unique()
order_items = order_items[order_items['order_id'].isin(valid_order_ids)].copy()
reviews = reviews[reviews['order_id'].isin(valid_order_ids)].copy()
payments = payments[payments['order_id'].isin(valid_order_ids)].copy()


## 5. 데이터셋 통합


In [251]:
# payments 집계 (한 주문에 여러 결제 수단이 있을 수 있음)
payments_agg = payments.groupby('order_id').agg({
    'payment_sequential': 'max',
    'payment_type': lambda x: ','.join(x.unique()),
    'payment_installments': 'max',
    'payment_value': 'sum'
}).reset_index()
payments_agg.columns = ['order_id', 'payment_sequential', 'payment_types', 
                        'payment_installments', 'payment_value_total']


In [252]:
# 데이터셋 통합
merged = orders.merge(order_items, on='order_id', how='left')
merged = merged.merge(products, on='product_id', how='left')
merged = merged.merge(category_trans, on='product_category_name', how='left')
merged['product_category_name_english'] = merged['product_category_name_english'].fillna('unknown')
merged = merged.merge(sellers, on='seller_id', how='left')
merged = merged.merge(reviews, on='order_id', how='left')
merged = merged.merge(customers, on='customer_id', how='left')
merged = merged.merge(payments_agg, on='order_id', how='left')


In [253]:
# 파생변수 생성

# 1. is_black_friday - 블랙 프라이데이 여부
# 브라질 블랙 프라이데이: 11월 마지막 금요일 (및 주변 기간)
# 2017: 11월 24일, 2018: 11월 23일
black_friday_dates = [
    ('2017-11-24', '2017-11-26'),  # 2017 블랙 프라이데이 ~ 일요일
    ('2018-11-23', '2018-11-25'),  # 2018 블랙 프라이데이 ~ 일요일
]

merged['is_black_friday'] = False
for start, end in black_friday_dates:
    mask = (merged['order_purchase_timestamp'] >= start) & (merged['order_purchase_timestamp'] <= end)
    merged.loc[mask, 'is_black_friday'] = True

# 2. is_carnival - 브라질 카니발 기간 여부
# 브라질 카니발: 부활절 전 40일경, 보통 2월 중순~3월 초
# 2017: 2월 24일~28일, 2018: 2월 9일~13일
carnival_dates = [
    ('2017-02-24', '2017-03-01'),  # 2017 카니발
    ('2018-02-09', '2018-02-14'),  # 2018 카니발
]

merged['is_carnival'] = False
for start, end in carnival_dates:
    mask = (merged['order_purchase_timestamp'] >= start) & (merged['order_purchase_timestamp'] <= end)
    merged.loc[mask, 'is_carnival'] = True

In [254]:
# 3. has_photos - 상품 사진 유무
merged['has_photos'] = merged['product_photos_qty'] > 0
print(f"3. has_photos: {merged['has_photos'].sum():,}건 ({merged['has_photos'].mean()*100:.2f}%)")

# 4. has_description - 상품 설명 유무
merged['has_description'] = merged['product_description_lenght'] > 0
print(f"4. has_description: {merged['has_description'].sum():,}건 ({merged['has_description'].mean()*100:.2f}%)")

# 5. description_length - 상품 설명 길이 (이미 존재하는 컬럼 활용)
merged['description_length'] = merged['product_description_lenght'].fillna(0).astype(int)
print(f"5. description_length: 평균 {merged['description_length'].mean():.1f}자, 최대 {merged['description_length'].max():,}자")


3. has_photos: 64,304건 (98.57%)
4. has_description: 64,304건 (98.57%)
5. description_length: 평균 774.3자, 최대 3,988자


In [255]:
# 7. has_text_review - 텍스트 리뷰 유무
merged['has_text_review'] = merged['review_comment_message'].notna() & (merged['review_comment_message'].str.len() > 0)

# 8. is_same_state - 판매자와 고객이 동일 주인지 여부
merged['is_same_state'] = merged['seller_state'] == merged['customer_state']

# 9. pg_processing_days - PG사 책임구간 (결제 승인까지 걸린 일수)
# order_approved_at - order_purchase_timestamp
merged['pg_processing_days'] = (
    (merged['order_approved_at'] - merged['order_purchase_timestamp']).dt.total_seconds() / (24 * 60 * 60)
).round(2)

# 10. seller_processing_days - 판매자 책임구간 (출고까지 걸린 일수)
# order_delivered_carrier_date - order_approved_at
merged['seller_processing_days'] = (
    (merged['order_delivered_carrier_date'] - merged['order_approved_at']).dt.total_seconds() / (24 * 60 * 60)
).round(2)

# 11. delivery_days - 물류사 책임구간 (배송까지 걸린 일수)
# order_delivered_customer_date - order_delivered_carrier_date
merged['delivery_days'] = (
    (merged['order_delivered_customer_date'] - merged['order_delivered_carrier_date']).dt.total_seconds() / (24 * 60 * 60)
).round(2)

# 12. is_logistics_fault - 물류사 과실 여부
# 예상 배송일보다 실제 고객 배송일이 늦으면 물류사 과실
merged['is_logistics_fault'] = merged['order_delivered_customer_date'] > merged['order_estimated_delivery_date']
print(f"12. is_logistics_fault (물류사 과실): {merged['is_logistics_fault'].sum():,}건 ({merged['is_logistics_fault'].mean()*100:.2f}%)")

# 13. seller_delay_days - 판매자 지연 일수
# (order_delivered_carrier_date - shipping_limit_date)
# 양수: shipping_limit_date를 넘겨서 물류사에 전달 (판매자 과실)
# 음수: shipping_limit_date 이전에 물류사에 전달 (정상)
merged['shipping_limit_date'] = pd.to_datetime(merged['shipping_limit_date'])
merged['seller_delay_days'] = (
    (merged['order_delivered_carrier_date'] - merged['shipping_limit_date']).dt.total_seconds() / (24 * 60 * 60)
).round(2)
print(f"13. seller_delay_days (판매자 지연 일수):")
print(f"    - 평균: {merged['seller_delay_days'].mean():.2f}일")
print(f"    - 양수(지연, 과실): {(merged['seller_delay_days'] > 0).sum():,}건")
print(f"    - 음수(제때 전달): {(merged['seller_delay_days'] <= 0).sum():,}건")

# 14. processing_days_diff - 카테고리 평균 대비 처리시간 차이
# (해당 주문 처리시간 - 카테고리 평균 처리시간)
# 양수: 평균보다 느림 (나쁨, 지연), 음수: 평균보다 빠름 (좋음)
category_avg = merged.groupby('product_category_name_english')['seller_processing_days'].transform('mean')
merged['processing_days_diff'] = (merged['seller_processing_days'] - category_avg).round(2)


12. is_logistics_fault (물류사 과실): 2,509건 (3.85%)
13. seller_delay_days (판매자 지연 일수):
    - 평균: -3.34일
    - 양수(지연, 과실): 5,590건
    - 음수(제때 전달): 59,650건


## 6. 최종 데이터셋 검증


In [256]:
# 최종 결측치 확인
print("=" * 60)
print("최종 통합 데이터셋 결측치 현황")
print("=" * 60)

null_counts = merged.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

if len(null_counts) > 0:
    print(f"\n결측치가 있는 컬럼 ({len(null_counts)}개):")
    for col, cnt in null_counts.items():
        pct = cnt / len(merged) * 100
        print(f"  {col}: {cnt:,}건 ({pct:.2f}%)")
else:
    print("결측치 없음")


최종 통합 데이터셋 결측치 현황

결측치가 있는 컬럼 (10개):
  review_comment_message: 37,380건 (57.30%)
  product_category_name: 936건 (1.43%)
  product_name_lenght: 936건 (1.43%)
  product_description_lenght: 936건 (1.43%)
  product_photos_qty: 936건 (1.43%)
  product_weight_g: 936건 (1.43%)
  product_length_cm: 936건 (1.43%)
  product_height_cm: 936건 (1.43%)
  product_width_cm: 936건 (1.43%)
  review_score: 390건 (0.60%)


In [257]:
# 최종 결측치 제거 (통합 후 발생한 결측치)
# 예외: review 관련 컬럼, product 관련 컬럼은 결측치 허용

# 결측치 제거에서 예외로 할 컬럼
exclude_cols = [
    # review 관련
    'review_score', 'review_comment_message',
    # product 관련
    'product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght',
    'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 
    'product_width_cm', 'product_category_name_english'
]

# 예외 컬럼을 제외한 나머지 컬럼에서만 결측치 검사
check_cols = [col for col in merged.columns if col not in exclude_cols]

before = len(merged)
merged = merged.dropna(subset=check_cols)
print(f"\n최종 결측치 제거: {before:,} -> {len(merged):,} ({before - len(merged):,}건 제거)")
print(f"  (review/product 관련 컬럼 결측치는 허용)")

# 불필요한 컬럼 제거
# 1. zip_code_prefix (주/도시는 유지)
# 2. product_category_name (영문 버전 사용)
cols_to_drop = [
    # 주소 관련
    'seller_zip_code_prefix',
    'customer_zip_code_prefix',
    # 카테고리 (영문 버전 사용)
    'product_category_name'
]

# 실제 존재하는 컬럼만 제거
existing_cols = [col for col in cols_to_drop if col in merged.columns]
merged = merged.drop(columns=existing_cols)

print(f"\n불필요한 컬럼 제거: {len(existing_cols)}개")
print(f"  제거된 컬럼: {existing_cols}")



최종 결측치 제거: 65,240 -> 65,240 (0건 제거)
  (review/product 관련 컬럼 결측치는 허용)

불필요한 컬럼 제거: 3개
  제거된 컬럼: ['seller_zip_code_prefix', 'customer_zip_code_prefix', 'product_category_name']


In [258]:
# review_score 결측치 제거 (리뷰가 없는 주문 제거)
print("=" * 60)
print("리뷰 없는 주문 제거")
print("=" * 60)

before_count = len(merged)
merged = merged.dropna(subset=['review_score'])
after_count = len(merged)

print(f"\n제거 전: {before_count:,}건")
print(f"제거 후: {after_count:,}건")
print(f"제거된 행: {before_count - after_count:,}건 (리뷰가 작성되지 않은 주문)")


리뷰 없는 주문 제거

제거 전: 65,240건
제거 후: 64,850건
제거된 행: 390건 (리뷰가 작성되지 않은 주문)


In [259]:
# 최종 데이터셋 정보
print("=" * 60)
print("최종 통합 데이터셋 정보")
print("=" * 60)
print(f"\n행 수: {len(merged):,}")
print(f"컬럼 수: {len(merged.columns)}")
print(f"\n컬럼 목록:")
for i, col in enumerate(merged.columns, 1):
    print(f"  {i:2d}. {col}")


최종 통합 데이터셋 정보

행 수: 64,850
컬럼 수: 46

컬럼 목록:
   1. order_id
   2. customer_id
   3. order_status
   4. order_purchase_timestamp
   5. order_approved_at
   6. order_delivered_carrier_date
   7. order_delivered_customer_date
   8. order_estimated_delivery_date
   9. order_item_id
  10. product_id
  11. seller_id
  12. shipping_limit_date
  13. price
  14. freight_value
  15. product_name_lenght
  16. product_description_lenght
  17. product_photos_qty
  18. product_weight_g
  19. product_length_cm
  20. product_height_cm
  21. product_width_cm
  22. product_category_name_english
  23. seller_city
  24. seller_state
  25. review_score
  26. review_comment_message
  27. customer_unique_id
  28. customer_city
  29. customer_state
  30. payment_sequential
  31. payment_types
  32. payment_installments
  33. payment_value_total
  34. is_black_friday
  35. is_carnival
  36. has_photos
  37. has_description
  38. description_length
  39. has_text_review
  40. is_same_state
  41. pg_processing_da

In [265]:
# 데이터 타입 확인
print("\n" + "=" * 60)
print("데이터 타입")
print("=" * 60)
print(merged.dtypes)



데이터 타입
order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
order_item_id                             int64
product_id                               object
seller_id                                object
shipping_limit_date              datetime64[ns]
price                                   float64
freight_value                           float64
product_name_lenght                     float64
product_description_lenght              float64
product_photos_qty                      float64
product_weight_g                        float64
product_length_cm                       float64
product_height_cm                       float64
product_width_cm                

In [266]:
# 최종 데이터셋 샘플
print("\n" + "=" * 60)
print("최종 데이터셋 샘플")
print("=" * 60)
merged.head().T



최종 데이터셋 샘플


,0,1,2,3,4
order_id,53cdb2fc8bc7dce0b6741e2150273451,47770eb9100c2d0c44946d9cf07ec65d,a4591c265e18cb1dcee52889e2d8acc3,6514b8ad8028c9f2cc2374ded245783f,76c6e866289321a7c93b82b54852dc33
customer_id,b0830fb4747a6c6d20dea0b8c802d7ef,41ce2a54c0b03bf3443c3d931a367089,503740e9ca751ccdda7ba28e9ab8f608,9bdf08b4b3b52b5526ff42d37d47f222,f54a9f0e6b351c431402b8461ea51999
order_status,delivered,delivered,delivered,delivered,delivered
order_purchase_timestamp,2018-07-24 20:41:37,2018-08-08 08:38:49,2017-07-09 21:57:05,2017-05-16 13:10:30,2017-01-23 18:29:09
order_approved_at,2018-07-26 03:24:27,2018-08-08 08:55:23,2017-07-09 22:10:13,2017-05-16 13:22:11,2017-01-25 02:50:47
order_delivered_carrier_date,2018-07-26 14:31:00,2018-08-08 13:50:00,2017-07-11 14:58:04,2017-05-22 10:07:46,2017-01-26 14:16:31
order_delivered_customer_date,2018-08-07 15:27:45,2018-08-17 18:06:29,2017-07-26 10:57:55,2017-05-26 12:55:51,2017-02-02 14:08:10
order_estimated_delivery_date,2018-08-13 00:00:00,2018-09-04 00:00:00,2017-08-01 00:00:00,2017-06-07 00:00:00,2017-03-06 00:00:00
order_item_id,1,1,1,1,1
product_id,595fac2a385ac33a80bd5114aec74eb8,aa4383b373c6aca5d8797843e5594415,060cb19345d90064d1015407193c233d,4520766ec412348b8d4caa5e8a18c464,ac1789e492dcd698c5c10b97a671243a


In [267]:
# 최종 요약
print("\n" + "=" * 60)
print("전처리 완료 요약")
print("=" * 60)
print(f"\n최종 데이터셋 shape: {merged.shape}")
print(f"고유 주문 수: {merged['order_id'].nunique():,}")
print(f"고유 고객 수: {merged['customer_id'].nunique():,}")
print(f"고유 판매자 수: {merged['seller_id'].nunique():,}")
print(f"고유 상품 수: {merged['product_id'].nunique():,}")
print(f"\n결측치: {merged.isnull().sum().sum()}개")


전처리 완료 요약

최종 데이터셋 shape: (64850, 46)
고유 주문 수: 56,696
고유 고객 수: 56,696
고유 판매자 수: 2,671
고유 상품 수: 23,032

결측치: 43493개


In [ ]:
# ML_olist 생성 (ML 모델링용 데이터셋)
ml_columns = [
    'order_id', 'order_approved_at', 'order_delivered_carrier_date',
    'seller_id', 'shipping_limit_date', 'product_category_name_english',
    'review_score', 'review_comment_message', 'has_text_review',
    'seller_processing_days', 'is_logistics_fault', 'seller_delay_days',
    'processing_days_diff', 'order_purchase_timestamp'
]
ML_olist = merged[ml_columns].copy()

In [ ]:
ML_olist.to_csv(f'{DATA_PATH}ML_olist.csv', index=False)
print(f"\n저장 완료: {DATA_PATH}ML_olist.csv")

In [268]:
merged_olist = merged

merged_olist.to_csv(f'{DATA_PATH}merged_olist.csv', index=False)
print(f"\n저장 완료: {DATA_PATH}merged_olist.csv")


저장 완료: Olist_DataSet/merged_olist.csv
